In [28]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [29]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])

32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [30]:


# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)



{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27


In [31]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  
  for w in words:
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr,  Ytr  = build_dataset(words[:n1])     # 80%
Xdev, Ydev = build_dataset(words[n1:n2])   # 10%
print(Xtr[:4])
print(Ytr[:4])
print(Xtr.shape[0])
print(Xtr.shape[0])
print(Xtr.shape[1])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
tensor([[ 0,  0,  0],
        [ 0,  0, 25],
        [ 0, 25, 21],
        [25, 21,  8]])
tensor([25, 21,  8,  5])
182625
182625
3


In [32]:
def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt-t.grad).abs().max().item()
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

    

In [33]:
n_embd = 10
n_hidden = 64
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embd), generator=g)
W1 = torch.randn((n_embd* block_size, n_hidden), generator=g) * (5/3) / ((n_embd * block_size) ** 0.5)
b1 = torch.randn(n_hidden, generator=g) * 0.1
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1
bngain = torch.randn((1, n_hidden), generator=g)* 1.0
bnbias = torch.randn((1, n_hidden), generator=g)* 0.1
parameters = [C, W1, W2, b1, b2,bngain, bnbias ]
for p in parameters:
  p.requires_grad = True





In [34]:
batch_size = 32
n = batch_size 

ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y

In [35]:
emb = C[Xb]
embcat = emb.view(emb.shape[0], -1)
hprebn = embcat @W1 + b1
#Batch norm layer
bnmeani = 1.0/n * hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff **2
bnvar = 1.0 / (n - 1) * (bndiff2).sum(0, keepdim=True)
bnvar_inv=(bnvar + 1e-5) **-0.5
bnraw = bndiff* bnvar_inv
hpreact = bngain * bnraw + bnbias

#activation function
h = torch.tanh(hpreact)

#linear layer
logits = h@W2 + b2

#now we do crossentropy
#Shape of  logits 32 * 27
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes
#The reason we subtract max from all values is that in the next step we perform exp
#and since exp grow very fast if we  have a high positive no the exp will reach inf
#Hence we subtract max which will make the highest no 1
#since e^0 is 1 and remaing no will be neg
counts =norm_logits.exp()
counts_sum = counts.sum(1, keepdim=True)#We take  for each row  we wil ahve one sum
#and we will divide each entry of corresponding row with its corresponding row sum

counts_sum_inv = counts_sum**-1 #This is same as 1/counts_sum
probs = counts * counts_sum_inv
logprobs = probs.log();
loss = -logprobs[range(n), Yb].mean()

for p in parameters:
    p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, 
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
         bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
         embcat, emb]:
  t.retain_grad()
loss.backward()
loss


tensor(3.3399, grad_fn=<NegBackward0>)

In [36]:
# #First we calc derivative of logprobs
# we can see that the loss is defined as loss = -logprobs(range(n), Yb).mean()
# basically what this means is this:
# since logprobs is 32*27 for each of the 32 rows we select one element corresponding to correct output(Yb)
# Then we add them together and take their mean
# We can simplify this using simple example
# lets suppose ls = (a + b + c) / 3
# then dls/da =1/3 dls/db = 1/3 and dls/dc = 1/3
# Our logprobs obeys this same example
# so for all the elements that are in the loss i.e 32 elements their derivative will be 1/n
# but what about the remaining 32*27 - 32 elements?
# since they do not participate in the loss their derivative will intuitively be 0
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0/n
cmp('logprobs', dlogprobs, logprobs)

#Now we calculate dprobs
# We know that we use chain rule
# so dloss/dprobs is dloss/dlogprobs * dlogprobs/dprobs
#We know dloss/dlogprobs -> dlogprobs
#We have to calculate dlogprobs/dprobs 
#logprobs is just the log of probs
#and derivative of log(x) -> 1/x
#so:
dprobs = (1.0/ probs) * dlogprobs
cmp('probs', dprobs, probs);



#Now we calculate dcounts_sum_inv
#since probs = count * count_sum_inv
#the derivative of a* b wrt a is b and wrt b is a
#Hence
dcounts_sum_inv= (counts * dprobs).sum(1, keepdim=True)#We have to sum this since we want the shape to be 1*32 since thats the shape of counts_sum_inv
dcounts = counts_sum_inv * dprobs
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)


#Now we calculate dcounts_sum
#since counts_sum_inv = counts_sum**-1
#hence when we take deriavative we have
dcounts_sum = (-counts_sum**-2) * dcounts_sum_inv
cmp('counts_sum', dcounts_sum, counts_sum)

#Next finding dcounts

#a11 a12 a3   ---->  b1(a1 + a2 + a3)
#a21 a22 a23  ----->b2(= a21 + a22 + a23)
#a31 a32 a33  ----->b3(= a31 + a32 + a33)
#counts_sum = counts.sum(1, keepdim=True)

dcounts += torch.ones_like(counts) * dcounts_sum
cmp('counts', dcounts, counts)


#Next we calc dnorm_logits
#counts =norm_logits.exp()
#Since derivative of e^x is e^x
dnorm_logits = norm_logits.exp() * dcounts
cmp('norm_logits', dnorm_logits, norm_logits)

#now we find for dlogits and dlogits_maxes
dlogits = dnorm_logits.clone();
dlogit_maxes = (-dnorm_logits).sum(1, keepdim=True)
cmp('logit_maxes', dlogit_maxes, logit_maxes)

#Next we again find dlogits since
#logit_maxes = logits.max(1, keepdim=True).values
#Hence
dlogits += F.one_hot(logits.max(1).indices, num_classes=logits.shape[1]) * dlogit_maxes
cmp('dlogits', dlogits, logits)


#Next we find dh dw2 and db2
dh = dlogits @W2.T
dw2 = h.T @ dlogits
db2 = dlogits.sum(0)
cmp('dh', dh, h);
cmp('dw2', dw2, W2);
cmp('db2', db2, b2)

dhpreact = (1.0 - h**2) * dh
cmp('dhpreact', dhpreact, hpreact)

#dbngain dbnraw and dbnbias
#hpreact = bngain * bnraw + bnbias
dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
dbnraw = (bngain * dhpreact)
dbnbias = dhpreact.sum(0, keepdim=True)
cmp('dbngain', dbngain, bngain);
cmp('dbnbias', dbnbias, bnbias)


#dbndiff & dbnvar_inv
#bnraw = bndiff* bnvar_inv
dbndiff = bnvar_inv * dbnraw 
dbnvar_inv =  (bndiff * dbnraw).sum(0, keepdim = True)
cmp('dbnvar_inv', dbnvar_inv, bnvar_inv)

#dbnvarr
#bnvar_inv=(bnvar + 1e-5) **-0.5
#for this we can use the power rule of derivative
dbnvar = -0.5 * ((bnvar + 1e-5) **-1.5 )* (dbnvar_inv)
cmp('dbnvar', dbnvar, bnvar)
#dbndiff2
#bnvar = 1.0 / (n - 1) * (bndiff2).sum(0, keepdim=True)
#if suppose we have array
# a11 a12
# a21 a22

# when we perform the operation we have
# b1 = (a11 + a12)*(1/n - 1)
# b2 = (a21 + a22) * (1/ n - 1)
dbndiff2 = (1.0 / (n - 1)) * torch.ones_like(bndiff2) *dbnvar
cmp('dbndiff2', dbndiff2, bndiff2)

#for dbndiff
#bndiff2 = bndiff **2
dbndiff += 2 * (bndiff) * dbndiff2
cmp('dbndiff', dbndiff, bndiff)


#dhprebn and dbnmeani
#bndiff = hprebn - bnmeani
#since we have - we have hint that we will use sum in the backprop
dhprebn = dbndiff.clone()
dbnmeani = (-dbndiff).sum(0)
#cmp('dhprebn', dhprebn, hprebn)
cmp('dbnmeani', dbnmeani, bnmeani)

#dhprebn
#bnmeani = 1.0/n * hprebn.sum(0, keepdim=True)
dhprebn += (1.0/n)*torch.ones_like(hprebn) *dbnmeani
cmp('dhprebn', dhprebn, hprebn)

# dembcat, dw1 and db1
#hprebn = embcat @W1 + b1
dembcat = dhprebn @W1.T
dW1 = embcat.T@dhprebn
db1 = dhprebn.sum(0)
cmp('dembcat', dembcat, embcat)
cmp('dW1', dW1, W1);
cmp('db1', db1, b1);

#demb
#embcat = emb.view(emb.shape[0], -1)
demb = dembcat.view(emb.shape)

cmp('demb', demb, emb)


#dc
#emb = C[Xb]
dC = torch.zeros_like(C)
for k in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
        ix  = Xb[k, j]
        dC[ix] +=demb[k, j]
cmp('dC', dC,C)
    

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
dlogits         | exact: True  | approximate: True  | maxdiff: 0.0
dh              | exact: True  | approximate: True  | maxdiff: 0.0
dw2             | exact: True  | approximate: True  | maxdiff: 0.0
db2             | exact: True  | approximate: True  | maxdiff: 0.0
dhpreact        | exact: False | approximate: True  | maxdiff: 4.656612873077393e-10
dbngain         | exact: False | approximate: True  | maxdiff: 3.725290298461914e-09
dbnbias         | exact: False | approximate: True  | maxdiff: 5.587935447692871e-09
dbnvar_i

In [37]:
print(C[Xb].shape)
print(Xb.shape)
print(C)

torch.Size([32, 3, 10])
torch.Size([32, 3])
tensor([[ 1.5674e+00, -2.3729e-01, -2.7385e-02, -1.1008e+00,  2.8588e-01,
         -2.9643e-02, -1.5471e+00,  6.0489e-01,  7.9136e-02,  9.0462e-01],
        [-4.7125e-01,  7.8682e-01, -3.2843e-01, -4.3297e-01,  1.3729e+00,
          2.9334e+00,  1.5618e+00, -1.6261e+00,  6.7716e-01, -8.4039e-01],
        [ 9.8488e-01, -1.4837e-01, -1.4795e+00,  4.4830e-01, -7.0730e-02,
          2.4968e+00,  2.4448e+00, -6.7006e-01, -1.2199e+00,  3.0314e-01],
        [-1.0725e+00,  7.2762e-01,  5.1114e-02,  1.3095e+00, -8.0220e-01,
         -8.5042e-01, -1.8068e+00,  1.2523e+00, -1.2256e+00,  1.2165e+00],
        [-9.6478e-01, -2.3211e-01, -3.4762e-01,  3.3244e-01, -1.3263e+00,
          1.1224e+00,  5.9641e-01,  4.5846e-01,  5.4011e-02, -1.7400e+00],
        [ 1.1560e-01,  8.0319e-01,  5.4108e-01, -1.1646e+00,  1.4756e-01,
         -1.0006e+00,  3.8012e-01,  4.7328e-01, -9.1027e-01, -7.8305e-01],
        [ 1.3506e-01, -2.1161e-01, -1.0406e+00, -1.5367e+00,  

In [38]:
#backward pass (short)
#loss = F.cross_entropy(logits, yb)

#dlogits
dlogits = F.softmax(logits, 1)
dlogits[range(n), Yb] -= 1
dlogits /=n
cmp('dlogits', dlogits, logits)


dlogits         | exact: False | approximate: True  | maxdiff: 5.3551048040390015e-09


In [39]:
#Now we calculate dhprebn given dhpreact
dhprebn = bngain*bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1)*bnraw*(dhpreact*bnraw).sum(0))

cmp('hprebn', dhprebn, hprebn) 


hprebn          | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10


In [ ]:
#Finally we now train this neural net!
n_embd = 10;
n_hidden = 200;
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, 10), generator=g)
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) *(5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden, generator=g)* 0.1

W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2  =torch.randn(vocab_size, generator=g) * 0.1

#batch norm params
bngain = torch.randn((1, n_hidden), generator=g) * 0.1 + 1
bnbias = torch.randn((1, n_hidden),generator=g ) * 0.1
parameters = [C, W1, b1, W2, b2, bngain, bnbias]
for p in parameters:
    p.requires_grad=True
    
    
    
    
    
#Now we perfrom the trainining
max_steps = 10000
batch_size = 54
n = batch_size
lossi = []


for i in range(max_steps):

   
    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix] 

    # forward pass
    emb = C[Xb] 
    embcat = emb.view(emb.shape[0], -1) 
    # Linear layer
    hprebn = embcat @ W1 + b1 # hidden layer pre-activation
    # BatchNorm layer
    bnmean = hprebn.mean(0, keepdim=True)
    bnvar = hprebn.var(0, keepdim=True, unbiased=True)
    bnvar_inv = (bnvar + 1e-5)**-0.5
    bnraw = (hprebn - bnmean) * bnvar_inv
    hpreact = bngain * bnraw + bnbias
    # Non-linearity
    h = torch.tanh(hpreact) # hidden layer
    logits = h @ W2 + b2 # output layer
    loss = F.cross_entropy(logits, Yb) # loss function

    
    for p in parameters:
      p.grad = None
    


    dlogits = F.softmax(logits, 1)
    dlogits[range(n), Yb] -= 1
    dlogits /= n
    # 2nd layer backprop
    dh = dlogits @ W2.T
    dW2 = h.T @ dlogits
    db2 = dlogits.sum(0)
    # tanh
    dhpreact = (1.0 - h**2) * dh
    # batchnorm backprop
    dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
    dbnbias = dhpreact.sum(0, keepdim=True)
    dhprebn = bngain*bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1)*bnraw*(dhpreact*bnraw).sum(0))
    # 1st layer
    dembcat = dhprebn @ W1.T
    dW1 = embcat.T @ dhprebn
    db1 = dhprebn.sum(0)
    # embedding
    demb = dembcat.view(emb.shape)
    dC = torch.zeros_like(C)
    for k in range(Xb.shape[0]):
      for j in range(Xb.shape[1]):
        ix = Xb[k,j]
        dC[ix] += demb[k,j]
    grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias]
    lr = 0.01 if i < 100000 else 0.001
    for p, grad in zip(parameters, grads):
     
      p.data += -lr * grad 
    

    
    if i % 1000 == 0: 
      print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())

      0/  10000: 3.7179
   1000/  10000: 2.5704
   2000/  10000: 2.3603
   3000/  10000: 2.8123
   4000/  10000: 2.3381
   5000/  10000: 2.6237
   6000/  10000: 2.4276
   7000/  10000: 2.3825
   8000/  10000: 2.4691
   9000/  10000: 2.1297
